In [ ]:
#VECTOR AUTO REGRESSION

In [29]:
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from statsmodels.tsa.vector_ar.var_model import VAR


In [10]:
data_df = pd.read_csv("Joined_data.csv",  delimiter=',')

In [11]:
data_df.head()

,Unnamed: 0,Date,AverageTemperature,AverageTemperatureUncertainty,COUNTRY,YEAR,MONTH,CAUSE_MODE,EQ_MAGNI_MEAN,ID_FIRST,...,ECLIPSE_TYPE,MAGNITUD,LATITUDE_y,LONGITUDE_y,DURATION_y,EVENT_y,DATE_STRIN,PLANET_POSITION,Lunar_Eclipse,Solar_Eclipse
0,0,1855-05-01,7.152,1.296,Chile,1855,5,NaN,NaN,NaN,...,P,0.7624,62.9N,16.6E,NaN,solar_eclipse,NaN,NaN,1,1
1,1,1855-06-01,4.257,1.641,Chile,1855,6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,2,1855-07-01,4.113,1.313,Chile,1855,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,3,1855-08-01,5.508,1.168,Chile,1855,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,4,1855-09-01,6.684,1.153,Chile,1855,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [12]:
data_df.columns

Index(['Unnamed: 0', 'Date', 'AverageTemperature',
       'AverageTemperatureUncertainty', 'COUNTRY', 'YEAR', 'MONTH',
       'CAUSE_MODE', 'EQ_MAGNI_MEAN', 'ID_FIRST', 'DEATHS_MEAN', 'DEATHS_MAX',
       'DEATHS_STD', 'DATE_FIRST', 'Event_Tsunami', 'Event_Earthquake',
       'DATE_STRIN_x', 'LATITUDE_x', 'LONGITUDE_x', 'PENUMBRAL',
       'PARTIAL ECLIPSE', 'DURATION_x', 'EVENT_x', 'DATE_STRIN_y',
       'ECLIPSE_TYPE', 'MAGNITUD', 'LATITUDE_y', 'LONGITUDE_y', 'DURATION_y',
       'EVENT_y', 'DATE_STRIN', 'PLANET_POSITION', 'Lunar_Eclipse',
       'Solar_Eclipse'],
      dtype='object')

In [20]:
col_for_model_lst = [
    'AverageTemperature',
    'AverageTemperatureUncertainty',
    'COUNTRY',
    'CAUSE_MODE',
    'EQ_MAGNI_MEAN',
    'DEATHS_MEAN',
    'DEATHS_MAX',
    'DEATHS_STD',
    'Event_Tsunami',
    'Event_Earthquake',
    'PENUMBRAL',
    #'PARTIAL ECLIPSE',
    #'ECLIPSE_TYPE',
    'MAGNITUD',
    #'PLANET_POSITION',
    'Lunar_Eclipse',
    'Solar_Eclipse'
]

In [21]:
mdata = data_df[col_for_model_lst]
mdata.head()

,AverageTemperature,AverageTemperatureUncertainty,COUNTRY,CAUSE_MODE,EQ_MAGNI_MEAN,DEATHS_MEAN,DEATHS_MAX,DEATHS_STD,Event_Tsunami,Event_Earthquake,PENUMBRAL,MAGNITUD,Lunar_Eclipse,Solar_Eclipse
0,7.152,1.296,Chile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,346.1,0.7624,1,1
1,4.257,1.641,Chile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,4.113,1.313,Chile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,5.508,1.168,Chile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,6.684,1.153,Chile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [22]:
mdata.shape

(11738, 14)

In [23]:
# Build my model for one country
#mdata.COUNTRY.unique()
mdata = mdata[mdata.COUNTRY=="Chile"]

mdata = mdata.drop("COUNTRY", axis=1)
display(mdata.head())
mdata.shape

,AverageTemperature,AverageTemperatureUncertainty,CAUSE_MODE,EQ_MAGNI_MEAN,DEATHS_MEAN,DEATHS_MAX,DEATHS_STD,Event_Tsunami,Event_Earthquake,PENUMBRAL,MAGNITUD,Lunar_Eclipse,Solar_Eclipse
0,7.152,1.296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,346.1,0.7624,1,1
1,4.257,1.641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,4.113,1.313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,5.508,1.168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,6.684,1.153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


(1960, 13)

In [24]:
mdata.dtypes

AverageTemperature               float64
AverageTemperatureUncertainty    float64
CAUSE_MODE                       float64
EQ_MAGNI_MEAN                    float64
DEATHS_MEAN                      float64
DEATHS_MAX                       float64
DEATHS_STD                       float64
Event_Tsunami                    float64
Event_Earthquake                 float64
PENUMBRAL                        float64
MAGNITUD                         float64
Lunar_Eclipse                      int64
Solar_Eclipse                      int64
dtype: object

In [26]:
mdata.index = pd.DatetimeIndex(data_df[data_df.COUNTRY=="Chile"]["Date"])

In [31]:
mdata.head()

,AverageTemperature,AverageTemperatureUncertainty,CAUSE_MODE,EQ_MAGNI_MEAN,DEATHS_MEAN,DEATHS_MAX,DEATHS_STD,Event_Tsunami,Event_Earthquake,PENUMBRAL,MAGNITUD,Lunar_Eclipse,Solar_Eclipse
Date,,,,,,,,,,,,,
1855-05-01,7.152,1.296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,346.1,0.7624,1,1
1855-06-01,4.257,1.641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1855-07-01,4.113,1.313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1855-08-01,5.508,1.168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1855-09-01,6.684,1.153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [36]:
# We cannot have nan values -- fill-in nan values with default value
mdata = mdata.fillna(0)

mdata.head()

,AverageTemperature,AverageTemperatureUncertainty,CAUSE_MODE,EQ_MAGNI_MEAN,DEATHS_MEAN,DEATHS_MAX,DEATHS_STD,Event_Tsunami,Event_Earthquake,PENUMBRAL,MAGNITUD,Lunar_Eclipse,Solar_Eclipse
Date,,,,,,,,,,,,,
1855-05-01,7.152,1.296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,346.1,0.7624,1,1
1855-06-01,4.257,1.641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0,0
1855-07-01,4.113,1.313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0,0
1855-08-01,5.508,1.168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0,0
1855-09-01,6.684,1.153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0,0


In [47]:
#data = np.log(mdata).replace([np.inf,-np.inf],0).diff().dropna()
data = mdata

In [48]:
# make a VAR model
model = VAR(data)

C:\Users\imypo\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'


In [49]:
results = model.fit()  # in the example, this is the desired lag order

In [50]:
results.summary()

LinAlgError: 3-th leading minor of the array is not positive definite

In [9]:
data = pd.read_csv("Joined_data.csv",  delimiter=',', parse_dates=['Date'], index_col="Date")

In [5]:
data.head()

,Unnamed: 0,AverageTemperature,AverageTemperatureUncertainty,COUNTRY,YEAR,MONTH,CAUSE_MODE,EQ_MAGNI_MEAN,ID_FIRST,DEATHS_MEAN,...,ECLIPSE_TYPE,MAGNITUD,LATITUDE_y,LONGITUDE_y,DURATION_y,EVENT_y,DATE_STRIN,PLANET_POSITION,Lunar_Eclipse,Solar_Eclipse
Date,,,,,,,,,,,,,,,,,,,,,
1855-05-01,0,7.152,1.296,Chile,1855,5,NaN,NaN,NaN,NaN,...,P,0.7624,62.9N,16.6E,NaN,solar_eclipse,NaN,NaN,1,1
1855-06-01,1,4.257,1.641,Chile,1855,6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1855-07-01,2,4.113,1.313,Chile,1855,7,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1855-08-01,3,5.508,1.168,Chile,1855,8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1855-09-01,4,6.684,1.153,Chile,1855,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [6]:
def testing_stationary(timeseries):
    # Determing rolling statistics
    roll_mean = timeseries.rolling(31).mean()
    roll_std = timeseries.rolling(31).std()

    # Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue', label='Original')
    mean = plt.plot(roll_mean, color='red', label='Rolling Mean')
    std = plt.plot(roll_std, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    #plt.show(block=False)

    

In [7]:
# Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')

    

IndentationError: unexpected indent (<ipython-input-7-7739ebd14a9a>, line 2)

In [ ]:
# The number of lags is chosen to minimise the Akaike's Information Criterion (AIC)
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])

    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

    print('\n\n')

   

In [ ]:
 # Perform KPSS test:
    print('Results of KPSS Test:')

    kpsstest = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic', 'p-value', 'Lags Used'])

    for key, value in kpsstest[3].items():
        kpss_output['Critical Value (%s)' % key] = value
    print(kpss_output)




In [ ]:
testing_stationary(data["Depth"])
# testing_stationary(data["Magnitude"])
# testing_stationary(data["Latitude"])
# testing_stationary(data["Longitude"])



In [ ]:
# Make data stationary
# 1st difference
df_differenced = data.diff().dropna()
testing_stationary(df_differenced["Depth"])
testing_stationary(df_differenced["Magnitude"])
testing_stationary(df_differenced["Latitude"])
testing_stationary(df_differenced["Longitude"])



In [ ]:
# Train-test part
train, test = df_differenced.iloc[:23158, :], data.iloc[23158:, :]



In [ ]:
# Fit the model
model = VAR(endog=train.astype(float), freq=None)
model_fit = model.fit()



In [ ]:
# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(test))

cols = data.columns
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,4):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

for i in cols:
    print('RMSE value for', i, 'is : ', sqrt(mean_squared_error(pred[i], test[i])))

p_mag = pd.DataFrame(test['Magnitude'])
p_mag['pred'] = pred['Magnitude'].values
p_mag.plot(legend=True)
plt.show()



In [ ]:
rmse_var_pred = sqrt(mean_squared_error(p_mag['Magnitude'], p_mag['pred']))
print("RMSE", rmse_var_pred)
var_score = r2_score(p_mag['Magnitude'], p_mag['pred'])
print("r2 score ", var_score)
var_ms = mean_squared_error(p_mag['Magnitude'], p_mag['pred'])
print("mean squared error ", var_ms)
var_m = mean_absolute_error(p_mag['Magnitude'], p_mag['pred'])
print("mean absolute error ", var_m)